# Scraping beer review site 

In [1]:
# Import dependencies for data wrangling  
import pandas as pd 
import numpy as np
import requests 
import json
from pprint import pprint

In [2]:
# Import dependencies for scraping
import pymongo
from time import sleep
from bs4 import BeautifulSoup

from splinter import Browser 
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Open the Chrome Driver Browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\pooja\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [4]:
# Read in CSV
beer_df = pd.read_csv("../../Data/beer.gz", encoding="ISO-8859-1")
beer_df.head()

,Unnamed: 0,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,0,10325,Vecchio Birraio,2009-02-16 20:57:03,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,1,10325,Vecchio Birraio,2009-03-01 13:44:57,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,2,10325,Vecchio Birraio,2009-03-01 14:10:04,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,3,10325,Vecchio Birraio,2009-02-15 19:12:25,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,4,1075,Caldera Brewing Company,2010-12-30 18:53:26,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [5]:
# Drop the "unnamed: 0" column 
beer_df = beer_df.drop(['Unnamed: 0'], axis = 1)

In [6]:
beer_df.drop_duplicates(subset = "brewery_id", keep = "first", inplace = True)

In [7]:
brewery_ids = beer_df["brewery_id"].unique()
type(brewery_ids)
len(brewery_ids)

5230

In [ ]:
assert False

In [8]:
address_list = list()
count = 0

for brewery_id in brewery_ids:
    
        url = "https://www.beeradvocate.com/beer/profile/" + str(brewery_id) + "/"
        browser.visit(url)
        sleep(1)

        html = browser.html
        soup = BeautifulSoup(html, 'lxml')
        
        
        try:

            browser.links.find_by_partial_text('map').click()
            browser.windows[1].is_current = True
            sleep(2)

            html = browser.html 
            soup = BeautifulSoup(html, 'lxml')

            input_tag = soup.find(id = "searchboxinput")
            output = input_tag['value']
            #print(output)

            address_list.append({
                "brewery_id": brewery_id,
                "address": output
            })

            count += 1
            print(count)

            browser.windows[1].close()
            # browser.visit(url) 

        except:
            pass
        sleep(2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=86.0.4240.111)


In [9]:
address_list

[{'brewery_id': 10325,
  'address': 'Via Caselle, 87, Marsango Di Campo San Martino, IT, 35010'},
 {'brewery_id': 1075, 'address': '590 Clover Lane, Ashland, OR, US, 97520'},
 {'brewery_id': 163, 'address': 'Antwoordnummer 7181, Amsterdam, NL, 1000'},
 {'brewery_id': 1454, 'address': '840 E 65th St, Indianapolis, IN, US, 46220'},
 {'brewery_id': 850, 'address': '21 West Bay St., Savannah, GA, US, 31401'},
 {'brewery_id': 14879,
  'address': '1680 E Waterloo Rd, Ste F, Akron, OH, US, 44306-4523'},
 {'brewery_id': 7345, 'address': 'Kallenfelser Str. 2-4, Kirn, DE, 55606'},
 {'brewery_id': 8802,
  'address': 'Unit 2, Ty Mawr, Tan Y Graig Road, Llysfaen, Colwyn Bay, GB7, UK, LL29 8UE'},
 {'brewery_id': 1386,
  'address': '1938 Pacific Avenue, Tacoma, WA, US, 98402-3110'},
 {'brewery_id': 9020, 'address': '900 River Bluff Dr, Madison, TN, US, 37115'},
 {'brewery_id': 20658,
  'address': '3006 Judson St, Ste 110, Gig Harbor, WA, US, 98335-1226'},
 {'brewery_id': 15636, 'address': '140 Hill S

In [12]:
address_df = pd.DataFrame(list(address_list), columns = ['brewery_id', 'address'])
address_df

,brewery_id,address
0,10325,"Via Caselle, 87, Marsango Di Campo San Martino..."
1,1075,"590 Clover Lane, Ashland, OR, US, 97520"
2,163,"Antwoordnummer 7181, Amsterdam, NL, 1000"
3,1454,"840 E 65th St, Indianapolis, IN, US, 46220"
4,850,"21 West Bay St., Savannah, GA, US, 31401"
...,...,...
362,27645,"156 Society Dr, Telluride, CO, US, 81435-8903"
363,273,"195 Ottley Drive NE, Atlanta, GA, US, 30324"
364,877,"2 Graphics Dr, Ewing, NJ, US, 08628"
365,11524,"Neradice 369, Uherský Brod, CZ, 688 16"


In [14]:
# Export the census data to csv to load to database
address_df.to_csv('first_set_address.csv', index=False)

In [ ]:
# extract unique breweries and beer names
brewery_beer_df = beer_df.loc[:,["brewery_id", "beer_beerid"]]
brewery_beer_df = brewery_beer_df.drop_duplicates(subset=["brewery_id", "beer_beerid"], keep=False)
brewery_beer = []

for row in brewery_beer_df.itertuples():
    brewery_beer.append(str(row.brewery_id) + "/" + str(row.beer_beerid))
    
# create webdriver object
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# scrape beer profile from beer advocate website for status
status_xpath = "/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[4]/div[1]/div[1]/div/div[2]/dl/dd[9]/span"
beer_xpath = "/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/input[2]"
beer_status = []
count = 0
for i in range(0, len(brewery_beer)):
    try:
        url = "https://www.beeradvocate.com/beer/profile/" + brewery_beer[i]
        browser.visit(url)
        status = browser.find_by_xpath(status_xpath).text
        beer = brewery_beer[i]
        beer_id = beer[beer.find("/")+1:]
        beer_status.append(beer_id)
        beer_status.append(status)
        print(beer_id)
        count +=1
        print(count)
    except:
        next

In [ ]:
# Page URL that we are going to scrape
url = "https://www.beeradvocate.com/beer/profile/" + str(brewery_ids[0]) + "/"
print(url)
browser.visit(url) 

# Convention to let the script we are running sleep to allow everything to load 
sleep(0.5)

In [ ]:
# Interact with the map link
browser.links.find_by_partial_text('map').click()

In [ ]:
browser.windows[1].is_current = True
html = browser.html 
soup = BeautifulSoup(html, 'lxml')

In [ ]:
input_tag = soup.find(id = "searchboxinput")
input_tag

In [ ]:
browser.windows[1].close()

In [ ]:
def listToString(s):
    
    address_string = ""
    
    for word in s:
        address_string += " " + word
    return address_string
 